In [38]:
# import jsonlines
# import faiss
# import torch
# from transformers import AutoTokenizer, AutoModel
# import numpy as np
# import os
# from huggingface_hub import login



In [39]:
# # !pip install huggingface_hub
# # !pip install ipywidgets
# login(os.getenv('hf_token'))

In [40]:
# !pip install langchain
# !pip install -U langchain-community
# !pip install -U langchain-ollama



In [41]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, AutoModelForMaskedLM
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM
import json
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModel
import numpy
from tqdm import tqdm



## Loading Encoder( & Retriever)

In [42]:
# 加載 Dragon 模型的 Tokenizer 和 Model
# Load model directly

# tokenizer = AutoTokenizer.from_pretrained("facebook/dragon-plus-query-encoder")
# context_encoder = AutoModel.from_pretrained('facebook/dragon-plus-query-encoder')


### ChatQA論文中使用的多輪檢索器

In [43]:

tokenizer = AutoTokenizer.from_pretrained("nvidia/dragon-multiturn-query-encoder")
context_encoder = AutoModel.from_pretrained('nvidia/dragon-multiturn-query-encoder')

# Loading corpus(PubMed)

In [44]:
# #absolute path
# corpus_path = r"..\corpus\pubmed24n0002.jsonl"


# # 讀取 JSONL 格式的文件，將每一行轉換為字典，並提取其中的 'content' 字段
# def load_corpus(file_path):
#     documents = []
#     with open(file_path, 'r') as file:
#         for line in file:
#             data = json.loads(line)
#             # 假設使用 'content' 或 'contents' 進行嵌入
#             text = data.get('contents')
#             if text:
#                 documents.append(text)
#     return documents

# # 加載你的corpus文件
# corpus = load_corpus(corpus_path)



In [45]:
# corpus[:2]

## Embedding function

In [46]:
# 轉換成向量
def embed_query(query: str):
    inputs = tokenizer(query, return_tensors="pt", truncation=True, max_length=512)
    outputs = context_encoder(**inputs)
     # 提取隱藏狀態的最後一層（hidden states），形狀為 (batch_size, sequence_length, hidden_size)
    last_hidden_state = outputs.last_hidden_state
    # 平均化隱藏層的輸出，這樣可以得到一個固定長度的向量（shape: [batch_size, hidden_size]）
    embedding = last_hidden_state.mean(dim=1).detach().cpu().numpy()
    return embedding

# Embedding

In [47]:
# 將每篇文檔轉換為嵌入向量
# from tqdm import tqdm
# corpus_embeddings = [embed_query(doc) for doc in tqdm(corpus, ascii=True, ncols=100, position=0, leave=True)]


檢視

In [48]:
# corpus_embeddings[0]

存到路徑

In [49]:
# import numpy as np

# save_path = '../embeddings/nvidia_pubmed24n0002.npy'

# # 保存嵌入向量到文件
# np.save(save_path, corpus_embeddings)


### 自定義embedding函數

In [50]:
from langchain.embeddings.base import Embeddings
import os

# 自定義 LangChain 的 Embeddings 類，使用你自己的 embed_query 函數
class CustomEmbeddings(Embeddings):
    def embed_documents(self, texts):
        return [embed_query(text).flatten().tolist() for text in texts]

    def embed_query(self, text):
        return embed_query(text).flatten().tolist()

# 初始化自定義嵌入模型
embedding_function = CustomEmbeddings()



### vectorstore目錄

In [51]:
import os
persist_directory = '../new_vectorstore'
os.makedirs(persist_directory, exist_ok=True)

## 重新載入chroma

In [52]:
# 載入已保存的 Chroma 數據庫
vectorstore = Chroma(
    embedding_function=embedding_function,  # 使用自定義的嵌入模型
    persist_directory=persist_directory,    # 指定存儲資料夾
)

# 現在可以進行查詢、檢索等操作


## Vectors into Chroma

In [53]:
# 假設 corpus 是你已經準備好的文本列表
# corpus_embeddings 已經由 embed_query 函數生成

# vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding_function)


# # 插入文檔和對應的嵌入向量
# ids = [f"doc1_{i}" for i in range(len(corpus))]
# vectorstore.add_texts(texts=corpus, ids=ids)

# # 持久化到磁碟
# vectorstore.persist()

# print(f"嵌入向量已保存到 {persist_directory}")


In [54]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 6})

In [55]:
# from evaluate import map_output_to_answer, format_question_with_options
# from utils import locate_answer, QADataset
# import ollama
# from datetime import datetime
# import numpy as np


# #HighLight
# def evaluate(dataset, model_name="llama3.1", wrong_pred_file=None,output_file=None):
#     flag = False
#     pred = []

#     answer_list = ["A", "B", "C", "D"]
#     answer2idx = {ans:i+1 for i, ans in enumerate(answer_list)}

#     total_len = len(dataset)

#     for q_idx in tqdm(range(total_len), desc="Processing Questions"):
#         question_only = dataset[q_idx]['question']
#         question_option_set = format_question_with_options(dataset[q_idx])

#         #相關文檔
#         #只納入問題，不納入選項
#         revelent_docs = retriever.invoke(question_option_set)
#         page_contents = "\n".join(f"{i+1}. {doc.page_content}" for i, doc in enumerate(revelent_docs))

#         # question = """You must output the 'option letter' only from the question below. 
#         # Example: A
#         # # Here is the question: {:s}
#         # # Below are the paragraphs for reference:\n
#         # # {:s}
#         #  """.format(question_option_set,page_contents)
#         # 使用模型產生答案
#         response = ollama.generate(
#             model=model_name,
#             prompt = """
#                 System: You must output the 'option letter' only from the question below. 
#                         Example: A
#                         Here are the references:
#                         {:s}

#                 User: {:s}

#                 Assistant:
#                 """.format(page_contents,question_option_set)
#         )
#         # response = ollama.generate(
#         #     model=model_name,
#         #     prompt=question,
#         # )
#         model_output = response['response'].strip().upper()
                
#         options = dataset[q_idx]['options']
#         #if model_output的len<2
#         if len(model_output) > 2:
#             # 如果model_output包含選項的內容，則直接取出選項，否則做re filter
#             pred_answer = next((opt for opt, content in options.items() if model_output.strip().lower() in content.strip().lower()), locate_answer(model_output))
#         else:
#             pred_answer = locate_answer(model_output)

#         # print('[Question]:', question)
#         # print('[Model]:', model_output)
#         # print('[pred_answer]:', pred_answer)
#         # print('truth:', dataset[q_idx]['answer'])
#         # Check if the model's output is valid
#         if pred_answer and pred_answer in answer_list:
#             pred.append(answer_list.index(pred_answer)+1)
#         else:
#             pred.append(-1)
        
#         map_output_to_answer(model_output)
#         # 所有問答
#         if pred_answer == dataset[q_idx]['answer']:
#             with open(output_file, 'a', encoding='utf-8') as f:
#                 f.write(f"[Question {q_idx+1}]: {dataset[q_idx]['question']}\n")
#                 f.write(f"[Options]: {dataset[q_idx]['options']}\n")
#                 f.write(f"[Reference]:\n {page_contents}\n")
#                 f.write(f"[Model Output]: {model_output}\n")
#                 f.write(f"[Predicted Answer]: {pred_answer}\n")
#                 f.write(f"[True Answer]: {dataset[q_idx]['answer']}\n\n")
        
#         # 錯誤預測的問題
#         if pred_answer != dataset[q_idx]['answer']:
#             with open(wrong_pred_file,'a',encoding='utf-8') as f:
#                 f.write(f"[Question {q_idx+1}]: {dataset[q_idx]['question']}\n")
#                 f.write(f"[Options]: {dataset[q_idx]['options']}\n")
#                 f.write(f"[Reference]:\n {page_contents}\n")
#                 f.write(f"[Model Output]: {model_output}\n")
#                 f.write(f"[Predicted Answer]: {pred_answer}\n")
#                 f.write(f"[True Answer]: {dataset[q_idx]['answer']}\n\n")

#     truth = [answer2idx[item['answer']] for item in dataset]
#     if len(pred) < len(truth):
#         truth = truth[:len(pred)]
#         flag = True

#     acc = (np.array(truth) == np.array(pred)).mean()
#     std = np.sqrt(acc * (1-acc) / len(truth))
#     return acc, std, flag



# 測試Medqa

In [56]:
# results_dir = r"C:\Users\ailab\Desktop\研究所\論文\baseline\records"
# dataset_name = 'medqa'
# dataset = QADataset(dataset_name)
# # dataset_name = ['medmcqa','medqa']
# # datasets = {key:QADataset(key) for key in dataset_name}
# scores = []
# current_time = datetime.now().strftime("%m-%d")

# save_dir = os.path.join(results_dir, "baseline")

# for dataset_name in dataset_name:
#     print("[{:s}] ".format(dataset_name), end="")
#     save_dir = os.path.join(results_dir, "baseline")
#     if os.path.exists(save_dir):
#         # 設置目錄和文件名
#         daily_dir = os.path.join(save_dir, 'rag', current_time)
#         # 檢查已有的編號資料夾，並自動分配新編號
#         idx = 1
#         while os.path.exists(f"{daily_dir}-{idx}"):
#             idx += 1
#         daily_dir = f"{daily_dir}-{idx}"
#         output_file = os.path.join(daily_dir, "full_record", f"{current_time}-{idx}.txt")
#         wrong_pred_file = os.path.join(daily_dir, "wrong_pred", f"{current_time}-{ idx}.txt")
#         # useless_ans_dir = os.path.join(daily_dir, "useless_ans", f"{current_time}.txt")

#         # 創建所需的所有目錄，exist_ok=True 允許目錄已存在
#         os.makedirs(os.path.dirname(output_file), exist_ok=True)
#         os.makedirs(os.path.dirname(wrong_pred_file), exist_ok=True)

#         acc, std, flag = evaluate(datasets[dataset_name], wrong_pred_file=wrong_pred_file, output_file=output_file)
#         scores.append(acc)
#         print("mean acc: {:.4f}; proportion std: {:.4f}".format(acc, std), end="")
#         if flag:
#             print(" (NOT COMPLETED)")
#         else:
#             print("")
#         with open(output_file, 'a', encoding='utf-8') as f:
#             f.write(f"{dataset_name} - mean acc: {acc:.4f}, std: {std:.4f}\n")
#             if flag:
#                 f.write(" (NOT COMPLETED)\n")
#             else:
#                 f.write("\n")
#     else:
#         print("NOT STARTED.")
#     if len(scores) > 0:
#         print("[Average] mean acc: {:.4f}".format(sum(scores) / len(scores)))
        


